In [ ]:
# from langchain_anthropic import ChatAnthropic
# import os

# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
# llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0.2, max_tokens=1024, anthropic_api_key=anthropic_api_key)

# # claude-3-sonnet-20240229


In [15]:
from IPython.display import display
from IPython.display import Markdown
from langchain_core.prompts import ChatPromptTemplate
import google.generativeai as genai
import textwrap

In [16]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [17]:
# models/embedding-001
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
import google.generativeai as genai
import os
api_key=os.environ.get("GOOGLE_API_KEY")


In [2]:
llm = genai.GenerativeModel(model_name = "gemini-pro")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [18]:
from langchain_google_genai import GoogleGenerativeAI

In [19]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
# print(
#     llm.invoke(
#         "What are some of the pros and cons of Python as a programming language?"
#     )
# )

In [5]:
from langchain.prompts import PromptTemplate

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "How much is 2+2?"
print(chain.invoke({"question": question}))

In [20]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("spark.pdf")

docs = loader.load()

In [21]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en")

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [22]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

In [23]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [25]:
response = retrieval_chain.invoke({"input": "what is lazy execution"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Lazy execution means that Spark will wait until the very last moment to execute your transformations. In Spark, instead of modifying the data quickly, we build up a plan of transformations that we would like to apply to our source data. Spark, by waiting until the last minute to execute the code, will compile this plan from your raw, DataFrame transformations, to an efficient physical plan that will run as efficiently as possible across the cluster. This provides immense benefits to the end user because Spark can optimize the entire data flow from end to end. An example of this might be “predicate pushdown” . If we build a large Spark job consisting of narrow dependencies, but specify a filter at the end that only requires us to fetch one row from our source data, the most efficient way to execute this is to access the single record that we need. Spark will actually optimize this for us by pushing the filter down automatically.
